In [7]:
import pandas as pd
import sqlite3

In [8]:
buses = pd.read_csv('C:/Users/chris/Downloads/Buses - Sheet1.csv')
passengers = pd.read_csv('C:/Users/chris/Downloads/passengers - Sheet1.csv')

In [9]:
conn = sqlite3.connect(':memory:')

buses.to_sql('buses', conn, index=False, if_exists='replace')
passengers.to_sql('passengers', conn, index=False, if_exists='replace')

15

In [10]:
next_highest = """
WITH NextHighestTimes AS (
    SELECT
        id,
        Origin,
        Destination,
        Departure,
        LAG(Departure) OVER (PARTITION BY Origin, Destination ORDER BY Departure) AS PreviousDeparture
    FROM
        buses
)
SELECT
    t1.id,
    t1.Origin,
    t1.Destination,
    COUNT(t2.id) AS passengercount
FROM
    NextHighestTimes t1
LEFT JOIN
    passengers t2
ON t2.Departure BETWEEN t1.PreviousDeparture AND t1.Departure
    AND t1.Origin=t2.Origin
    AND t1.Destination=t2.Destination 
GROUP BY
    t1.id, t1.origin, t1.destination
"""

In [11]:
results = pd.read_sql(next_highest, conn)

In [12]:
print(results)

   id Origin Destination  passengercount
0   1      A           B               0
1   2      A           B               1
2   3      A           B               1
3   4      B           C               0
4   5      B           C               1
5   6      B           C               0
6   7      B           C               5
7   8      B           A               0
8   9      B           A               2
9  10      B           A               5
